In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow

In [31]:
#Загрузка весов и конфигурационного файла
#Модифицированный файл cfg с изменениями для классов и размера фильтра
net = cv2.dnn.readNet("/content/drive/MyDrive/ConstructionYolo/weights/yolov3.weights", "/content/drive/MyDrive/ConstructionYolo/yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
#Целевой класс для обнаружения
classes = ["floor"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
#абор данных для тестирования
images_path = glob.glob("/content/drive/MyDrive/ConstructionYolo/test/*.jpg") 
for image in images_path:
    #Загрузка изображения и преобразование в нужный формат
    img = cv2.imread(image)
    scale_percent = 60
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (2, 2, 2), True, crop=False)
    net.setInput(blob)
    output_of_layer = net.forward(output_layers)
    #Отображение информации на экране
    class_ids = []
    confidences = []
    boxes = []
    num_detection=0 
    for output in output_of_layer:
        for detection in output: 
            #Используем порог
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]  
            if confidence > 0.3:
                num_detection=num_detection+1
                centroid_x = int(detection[0] * width)
                centeroid_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                #Прямоугольные координаты
                x = int(centroid_x - w / 2)
                y = int(centeroid_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    print("Общее количество этажей в здании=",num_detection)   
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)

    cv2_imshow(img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [32]:
pip freeze > requirements.txt